BASIC CIFAR 10 EXAMPLE

Install Keras just in case...

In [29]:
!pip3 install keras

Imports...

In [30]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD

from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator

Define batch size, number of epochs and number of classes


In [31]:
batch_size = 100
num_classes = 10
epochs = 75

Load MNIST and transform

In [32]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


DEFINE A DATA AUGMENTATION GENERATOR

> Bloque con sangría



In [33]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

DEF A BLOCK CONV + BN + GN

In [34]:
def CBGN(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))
  
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))  
  
  return model

Define the NN topology, a sequential model with 2 hidden layers

In [35]:
model = Sequential()

model=CBGN(model,64,x_train.shape[1:])
model=CBGN(model,64)
model.add(MaxPooling2D(pool_size=(2, 2)))

model=CBGN(model,128)
model=CBGN(model,128)
model.add(MaxPooling2D(pool_size=(2, 2)))

model=CBGN(model,256)
model=CBGN(model,256)
model=CBGN(model,256)
model.add(MaxPooling2D(pool_size=(2, 2)))

model=CBGN(model,512)
model=CBGN(model,512)
model=CBGN(model,512)
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_10 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
gaussian_noise_10 (GaussianN (None, 32, 32, 64)        0         
_________________________________________________________________
activation_14 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_11 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
gaussian_noise_11 (GaussianN (None, 32, 32, 64)       

Define an optimizer 

In [36]:
opt = SGD(lr=0.1, decay=1e-6)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


DEFINE A LEARNING RATE SCHEDULER

In [37]:
def scheduler(epoch):
    if epoch < 25:
        return .1
    elif epoch < 50:
        return 0.01
    else:
        return 0.001

# Callbacks
set_lr = LRS(scheduler)
es = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

Compile the model, define loss and link the optimizer

In [38]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Finally, train the model and evaluate over the test data

In [ ]:
history=model.fit(datagen.flow(x_train, y_train,batch_size=batch_size),
                  steps_per_epoch=len(x_train) / batch_size, 
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  callbacks=[set_lr, es],
                  verbose=1)

# Evaluate over test
score = model.evaluate(x_test, y_test, verbose=0)


print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/75
500/500 [==============================] - 74s 137ms/step - loss: 2.4755 - accuracy: 0.1854 - val_loss: 2.3078 - val_accuracy: 0.2018
Epoch 2/75
500/500 [==============================] - 66s 133ms/step - loss: 1.5849 - accuracy: 0.4069 - val_loss: 1.6138 - val_accuracy: 0.4422


In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()